#Synthesis Notes Evaluation

Notebook này thực hiện nhiệm vụ phân tích và đánh giá chất lượng của ghi chú y tế tổng hợp

## Evaluate Synthesis V1

### Prepare

In [ ]:
!pip install -q transformers datasets==3.4.1
!pip install sacremoses
!git clone https://github.com/chancholat/mimic.git

!mkdir /content/src
!gdown --folder https://drive.google.com/drive/folders/1b3tBwPYlPx0j-3pdJy4Na6wQBZUTJezv -O /content/src

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6

In [ ]:
import kagglehub
#Download the latest version.
kagglehub.dataset_download('chanhainguyen/thesis-data-process')

100%|██████████| 2.95G/2.95G [01:45<00:00, 30.1MB/s]

Extracting files...


'/root/.cache/kagglehub/datasets/chanhainguyen/thesis-data-process/versions/1'

In [ ]:
!mkdir mimic/data
!mv /root/.cache/kagglehub/datasets/chanhainguyen/thesis-data-process/versions/1/processed /content/mimic/data
# !mv /kaggle/input/thesis-data-process/processed /content/mimic/data

In [ ]:
!mkdir /content/syn_v1
!gdown --folder https://drive.google.com/drive/folders/1C50EBOlFEioaCG8AJrHWCsNh2IFPzQ65 -O /content/syn_v1 --remaining-ok

Retrieving folder contents
Processing file 1MYLP6LeTTmbtI_e_nyA6NVv5fsDo7stp result0_40_df.parquet
Processing file 144X-ChLwM73SPvFCVqsOe5i5f6SeJPxY result40_80_df.parquet
Processing file 1bfPwv-dbXtb8ncf8V0kXmGyO4ywtevSP result80_120_df.parquet
Processing file 1MiUGpoG6gjXHJG5ynI1Fa0oZXytQIGWe result120_160_df.parquet
Processing file 1kuG9xsxGCWpuR1mCnvtFEsZMLTADZ5vi result160_200_df.parquet
Processing file 1NEpJ4NUqXYLir-W1XkmaMkkCvOLlDWDL result200_240_df.parquet
Processing file 1FLcJtH7NTkVJabi8xol45YPuyiE_ILWz result240_280_df.parquet
Processing file 1YVuu2UVbuxXxxgJZNg9ocgs2Mejo7HVl result280_320_df.parquet
Processing file 11FCG1xuni9mU0AhrlbpEjS-phc4kVbER result320_360_df.parquet
Processing file 1S5Qd0DzSQmWq1S5_g7NWL-qVW3PXvqzb result360_400_df.parquet
Processing file 1WZfdXC6fZYvzVxRED-eHO3xFP1ML71UZ result400_440_df.parquet
Processing file 1rTlWKF2cfOLSx_5QHeikOdeycQExGe9h result440_480_df.parquet
Processing file 1ElaRd1TlY60fEj6XzeosAlsJhvlsQ1UC result480_520_df.parquet
Proc

#### Load MIMIC

In [ ]:
%cd /content/
from datasets import load_dataset
from pathlib import Path
# SIZE = 10
# mimic = load_dataset(str(dataset_path), split=f"train[:{SIZE}]", trust_remote_code=True)
dataset_path = Path("/content/mimic/mdace_inpatient_icd9_code.py")
mimic = load_dataset(str(dataset_path), trust_remote_code=True)

/content


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

#### ICD9 Hierarchy

In [ ]:
# Option 1: Clone the repository
%cd /content/
!git clone https://github.com/sirrice/icd9.git
%cd icd9

/content
Cloning into 'icd9'...
remote: Enumerating objects: 76, done.
remote: Total 76 (delta 0), reused 0 (delta 0), pack-reused 76 (from 1)
Receiving objects: 100% (76/76), 1.28 MiB | 3.62 MiB/s, done.
Resolving deltas: 100% (37/37), done.
/content/icd9


In [ ]:
# Point to the provided JSON file in the repo
# Need to modify the icd9.py file to use open() instead of file()
# Original icd9.py code uses file() which is deprecated in Python 3
# The fix is to replace file() with open() in the __init__ method of the ICD9 class
import inspect
import os

icd9_file_path = '/content/icd9/icd9.py'

with open(icd9_file_path, 'r') as f:
    lines = f.readlines()

modified_lines = []
for line in lines:
    if 'with file(codesfname, \'r\') as f:' in line:
        modified_lines.append(line.replace('with file(codesfname, \'r\') as f:', 'with open(codesfname, \'r\') as f:'))
    else:
        modified_lines.append(line)

with open(icd9_file_path, 'w') as f:
    f.writelines(modified_lines)

In [ ]:
from icd9 import ICD9
# Now load the ICD9 tree
tree = ICD9('/content/icd9/codes.json')
def get_siblings(code):
    node = tree.find(code)
    return [sib.code for sib in node.siblings] if node and node.siblings else []

def get_parent(code):
    node = tree.find(code)
    return node.parent.code if node and node.parent else None

#### Build common phrases

In [ ]:
import pandas as pd
from collections import Counter
from tqdm import tqdm

def extract_common_phrases(
    notes: list[str],
    lines_from_end: int = 10,
    min_phrase_len: int = 2,
    max_phrase_len: int = 50,   # NEW PARAMETER
    start_char: str = "",
    stop_char: str = "",
    top_k: int = 30
):
    """
    Scan through clinical notes to find commonly occurring ending phrases.

    Args:
        notes: List of clinical note texts.
        lines_from_end: How many lines from the end to extract per note.
        min_phrase_len: Minimum number of words in a phrase to count.
        max_phrase_len: Maximum number of words in a phrase to count.
        top_k: Number of top phrases to return.
    """
    counter = Counter()

    for note in tqdm(notes, desc="Scanning notes"):
        lines = note.strip().splitlines(keepends=True)
        tail_lines = lines[-lines_from_end:]  # Last N lines of the note

        for i, line in enumerate(tail_lines):
            phrase = line.strip(' ')
            word_count = len(phrase.split(' '))
            if min_phrase_len <= word_count <= max_phrase_len and tail_lines[max(0, i-1)].startswith(start_char) and phrase.endswith(stop_char):
                counter[phrase] += 1

    return counter.most_common(top_k)


In [ ]:
from datasets import load_dataset
from pathlib import Path

dataset_path = Path("/content/mimic/mdace_inpatient_icd9_code.py")
mimic = load_dataset(str(dataset_path), trust_remote_code=True)
notes = mimic["train"]["text"]

In [ ]:
discharge_rows = mimic['train'].filter(lambda x: x["note_type"] == "Discharge summary")
notes = discharge_rows["text"]
common_headings = extract_common_phrases(
      notes,
      lines_from_end=0,
      min_phrase_len=2,
      max_phrase_len=5,
      start_char='\n',
      stop_char=':\n',
      top_k=200
    )
print(len(common_headings))
for phrase, count in common_headings:
    print(f"{phrase.strip()} — {count} notes")

common_headings = [phrase for phrase, _ in common_headings]

Filter:   0%|          | 0/48074 [00:00<?, ? examples/s]

Scanning notes: 100%|██████████| 47904/47904 [00:10<00:00, 4502.29it/s]

200
Discharge Disposition: — 35615 notes
Brief Hospital Course: — 33803 notes
Discharge Diagnosis: — 33783 notes
Major Surgical or Invasive Procedure: — 33747 notes
Discharge Condition: — 33743 notes
Past Medical History: — 33668 notes
History of Present Illness: — 33637 notes
Social History: — 33110 notes
Physical Exam: — 33110 notes
Pertinent Results: — 33081 notes
Discharge Instructions: — 33013 notes
Medications on Admission: — 32913 notes
Followup Instructions: — 32842 notes
Family History: — 32721 notes
Discharge Medications: — 32297 notes
DISCHARGE DIAGNOSES: — 6617 notes
PAST MEDICAL HISTORY: — 5003 notes
DISCHARGE MEDICATIONS: — 4886 notes
[**Hospital 93**] MEDICAL CONDITION: — 3118 notes
DISCHARGE DIAGNOSIS: — 2442 notes
MEDICATIONS ON DISCHARGE: — 1850 notes
MEDICATIONS ON ADMISSION: — 1830 notes
Cranial Nerves: — 1528 notes
HOSPITAL COURSE: — 1411 notes
FINAL DIAGNOSIS: — 837 notes
CARE RECOMMENDATIONS: — 800 notes
DISCHARGE INSTRUCTIONS: — 751 notes
PAST SURGICAL HISTORY: 

### Compare Real vs Synthesis Note

In [ ]:
bid = 3

#### Prepare

In [ ]:
import pandas as pd
note_40 = pd.read_parquet('/content/syn_v1/result0_40_df.parquet')
note_40['text'][bid]

'Diagnosis Codes: 250.80 (Diabetes with other specified manifestations, type II or unspecified type, not stated as uncontrolled); 401.9 (Unspecified essential hypertension); 585.9 (Chronic kidney disease, unspecified); 403.00 (Hypertensive chronic kidney disease, malignant, with chronic kidney disease stage I through stage IV, or unspecified); V15.81 (Personal history of noncompliance with medical treatment, presenting hazards to health); 275.42 (Hypercalcemia); 584.9 (Acute kidney failure, unspecified); 595.0 (Acute cystitis); 241.0 (Nontoxic uninodular goiter); 285.21 (Anemia in chronic kidney disease) Procedure Codes: None Admission Date: [* * 2151-8-15 * *] Discharge Date: [* * 2151-8-22 * *] Date of Birth: [* * 2089-6-19 * *] Sex: F Service: MEDICINE Allergies: Penicillins / Sulfa (Sulfonamides) Attending: [* * First Name3 (LF) 1985 * *] Chief Complaint: Dysuria Major Surgical or Invasive Procedure: None History of Present Illness: 79 yo F with PMH of HTN, DM, CKD, h / o CVA, and 

In [ ]:
print(note_40['prompt'][bid])
print(note_40['augmented codes'][bid])
print(note_40['rare codes'][bid])

Diagnosis Codes: 250.80 ( Diabetes with other specified manifestations, type II or unspecified type, not stated as uncontrolled); 401.9 ( Unspecified essential hypertension); 585.9 ( Chronic kidney disease, unspecified); 403.00 ( Hypertensive chronic kidney disease, malignant, with chronic kidney disease stage I through stage IV, or unspecified); V15.81 ( Personal history of noncompliance with medical treatment, presenting hazards to health); 275.42 ( Hypercalcemia); 584.9 ( Acute kidney failure, unspecified); 595.0 ( Acute cystitis); 241.0 ( Nontoxic uninodular goiter); 285.21 ( Anemia in chronic kidney disease)
Procedure Codes: None


['250.80' '401.9' '585.9' '403.00' 'V15.81' '275.42' '584.9' '595.0'
 '241.0' '285.21']
['241.0']


#### Select rare diagnosis codes

In [ ]:
import pandas as pd
from collections import Counter

train_dx_codes = mimic['train']['diagnosis_codes']
train_proc_codes = mimic['train']['procedure_codes']

val_dx_codes = mimic['validation']['diagnosis_codes']
val_proc_codes = mimic['validation']['procedure_codes']

train_dx_code_count = Counter([code for sublist in train_dx_codes for code in sublist])
train_proc_code_count = Counter([code for sublist in train_proc_codes for code in sublist])

val_dx_code_count = Counter([code for sublist in val_dx_codes for code in sublist])
val_proc_code_count = Counter([code for sublist in val_proc_codes for code in sublist])

num_train_dx_code = len(train_dx_code_count)
num_train_proc_code = len(train_proc_code_count)
total_train_dx_code = sum(train_dx_code_count.values())
total_train_proc_code = sum(train_proc_code_count.values())

num_val_dx_code = len(val_dx_code_count)
num_val_proc_code = len(val_proc_code_count)
total_val_dx_code = sum(val_dx_code_count.values())
total_val_proc_code = sum(val_proc_code_count.values())

print(f"Number of train diagnosis codes: {num_train_dx_code}")
print(f"Number of train procedure codes: {num_train_proc_code}")
print(f"Total number of train diagnosis codes: {total_train_dx_code}")
print(f"Total number of train procedure codes: {total_train_proc_code}")
print()
print(f"Number of validation diagnosis codes: {num_val_dx_code}")
print(f"Number of validation procedure codes: {num_val_proc_code}")
print(f"Total number of validation diagnosis codes: {total_val_dx_code}")
print(f"Total number of validation procedure codes: {total_val_proc_code}")

Number of train diagnosis codes: 6741
Number of train procedure codes: 1971
Total number of train diagnosis codes: 549845
Total number of train procedure codes: 200621

Number of validation diagnosis codes: 2394
Number of validation procedure codes: 650
Total number of validation diagnosis codes: 23313
Total number of validation procedure codes: 5772


In [ ]:
import pandas as pd
from collections import Counter

# Flatten all codes
# It is important that we only synthesis the dianosis code,
# Because the procedure code is assigned based on how each specific desease is diagnosis

# Step 3: Assign to DataFrame
train_df = pd.DataFrame()
val_df = pd.DataFrame()
train_df["train_dx_codes"] = [codes for codes in train_dx_codes if isinstance(codes, list) and len(codes) > 0]
val_df["val_dx_codes"] = [codes for codes in val_dx_codes if isinstance(codes, list) and len(codes) > 0]
all_train_codes = [code for row in train_df["train_dx_codes"] for code in row]
all_val_codes = [code for row in val_df["val_dx_codes"] for code in row]
train_code_freq = Counter(all_train_codes)
val_code_freq = Counter(all_val_codes)

In [ ]:
import numpy as np


def get_percentiles(freq_values):
  # Compute quartiles
  q1 = np.percentile(freq_values, 25)   # First quartile (Q1)
  q2 = np.percentile(freq_values, 50)   # Second quartile (median, Q2)
  q3 = np.percentile(freq_values, 75)   # Third quartile (Q3)
  q87 = np.percentile(freq_values, 87.5)
  q90 = np.percentile(freq_values, 90)
  q93 = np.percentile(freq_values, 93)
  q96 = np.percentile(freq_values, 96)
  q99 = np.percentile(freq_values, 99)
  q4 = np.max(freq_values)              # Fourth quartile (max)

  print(f"Q1 (25th percentile): {q1}")
  print(f"Q2 (Median): {q2}")
  print(f"Q3 (75th percentile): {q3}")
  print(f"Q87.5 (87.5th percentile): {q87}")
  print(f"Q90 (90th percentile): {q90}")
  print(f"Q93 (93th percentile): {q93}")
  print(f"Q96 (96th percentile): {q96}")
  print(f"Q99 (99th percentile): {q99}")
  print(f"Q4 (Max): {q4}")
  print()
  return q87, q90, q93, q96, q99, q4, q3, q2, q1

q87, q90, q93, q96, q99, q4, q3, q2, q1 = get_percentiles(np.array(list(train_code_freq.values())))
_, val_q90, _, _, _, val_q4, val_q3, val_q2, val_q1 = get_percentiles(np.array(list(val_code_freq.values())))

Q1 (25th percentile): 2.0
Q2 (Median): 5.0
Q3 (75th percentile): 26.0
Q87.5 (87.5th percentile): 90.0
Q90 (90th percentile): 126.0
Q93 (93th percentile): 199.40000000000146
Q96 (96th percentile): 390.7999999999993
Q99 (99th percentile): 1290.6000000000022
Q4 (Max): 17963

Q1 (25th percentile): 1.0
Q2 (Median): 2.0
Q3 (75th percentile): 6.0
Q87.5 (87.5th percentile): 15.0
Q90 (90th percentile): 19.0
Q93 (93th percentile): 29.0
Q96 (96th percentile): 45.279999999999745
Q99 (99th percentile): 134.07000000000016
Q4 (Max): 735



In [ ]:
threshold = q90
val_threshold = val_q3
rare_codes = {code for code, freq in train_code_freq.items() if freq < threshold and val_code_freq[code] >= val_threshold}
frequent_codes = {code for code, freq in train_code_freq.items() if freq >= threshold}
anchor_sets = train_df[train_df["train_dx_codes"].apply(lambda codes: any(code in frequent_codes for code in codes))]["train_dx_codes"].tolist()
rare_code_sets = train_df[train_df["train_dx_codes"].apply(lambda codes: any(code in rare_codes for code in codes))]["train_dx_codes"].tolist()
only_rare_code_sets = train_df[train_df["train_dx_codes"].apply(lambda codes: all(code in rare_codes for code in codes))]["train_dx_codes"].tolist()
rare_frequent_code_sets = train_df[train_df["train_dx_codes"].apply(lambda codes: any(code in rare_codes for code in codes) and any(code in frequent_codes for code in codes))]["train_dx_codes"].tolist()
print("frequent codes len: ", len(frequent_codes))
print("rare codes len: ", len(rare_codes))
print("frequent sets len: ", len(anchor_sets))
print("rare sets len: ", len(rare_code_sets))
print("only rare sets len: ", len(only_rare_code_sets))
print("rare frequent sets len: ", len(rare_frequent_code_sets))


frequent codes len:  676
rare codes len:  124
frequent sets len:  47804
rare sets len:  8825
only rare sets len:  2
rare frequent sets len:  8795


#### Tracing real codes from augmented codes

In [ ]:
from typing_extensions import TypeVarTuple
from datasets import Dataset
from typing import Callable, List, Set

def find_sibling_code_entry(
    dataset: Dataset,
    diagnosis_col: str,
    original_codes: Set[str],
    rare_code: str
):
    frequent_codes = original_codes - {rare_code}
    sibling_codes = set(get_siblings(rare_code))
    parent_codes = set([get_parent(rare_code)])

    def is_match(example):
        codes = set(example[diagnosis_col])
        # if rare_code in codes:
        #     return False
        if not frequent_codes.issubset(codes):
            return False
        if not codes.intersection(sibling_codes) and not codes.intersection(parent_codes):
            return False
        return True

    matches = dataset.filter(is_match)
    return matches


In [ ]:
# original_codes = set(['282.2', '715.95', 'V12.51', '282.5', '288.3', '714.0', '241.0', '790.7', '018.96', '427.89', '238.72', '780.6', '599.0', '511.9', '041.4'])
# rare_code_in_set = [code for code in original_codes if code in rare_codes]
# rare_code = str(rare_code_in_set[1])
# print(rare_code_in_set)

In [ ]:
# Find entries that match the crteria
original_codes = set(note_40['augmented codes'][bid])
# rare_code = str(note_40['rare codes'][bid][0])
rare_code_in_set = [code for code in note_40['augmented codes'][bid] if code in rare_codes]
rare_code = rare_code_in_set[0]

matches = find_sibling_code_entry(mimic['train'], 'diagnosis_codes', original_codes, rare_code)
print(matches[0]['diagnosis_codes'])

Filter:   0%|          | 0/48074 [00:00<?, ? examples/s]

['482.0', '530.81', 'V12.71', '284.1', '571.5', '268.9', '518.5', '567.23', '285.8', '599.0', 'V10.07', '250.00', '333.2', '733.90', '286.9', '070.44', '995.92', '241.0', '276.6', 'V45.89', '780.39', '041.04', 'V46.11', 'V16.0', '785.52', '511.9', '038.49', '584.9']


In [ ]:
rare_code

'268.9'

In [ ]:
# print(find_match_real_code_sets(original_codes, mimic, rare_codes))

In [ ]:

# original_codes = set(['282.2', '715.95', 'V12.51', '282.5', '288.3', '714.0', '241.0', '790.7', '018.96', '427.89', '238.72', '780.6', '599.0', '511.9', '041.4'])
# extract_matched_note_id(original_codes, mimic, rare_codes)

#### Parsing sections

In [ ]:
import re
from typing import Dict, List

def extract_note_sections(full_text: str, known_headings: List[str], is_real=False) -> Dict[str, str]:
    # 1. Cut off everything before and including 'Procedure Codes: None'
    note_text = full_text
    if not is_real:
      split_marker = re.search(r'procedure codes:\s*none', full_text, re.IGNORECASE)
      if not split_marker:
          raise ValueError("Could not find 'Procedure Codes: None' marker.")
      note_text = full_text[split_marker.end():].strip()

    # 2. Normalize and prepare headings
    heading_patterns = [
        re.escape(h.strip()) for h in known_headings
    ]
    heading_regex = r'(' + '|'.join(heading_patterns) + r')\s*:?'  # support colon and no colon

    # 3. Find all heading matches
    matches = list(re.finditer(heading_regex, note_text, re.IGNORECASE))

    # 4. Extract sections
    sections = {}
    for i, match in enumerate(matches):
        heading = match.group(1).strip()
        start = match.end()
        end = matches[i+1].start() if i+1 < len(matches) else len(note_text)
        content = note_text[start:end].strip()
        sections[heading] = content

    return sections

In [ ]:
full_text = note_40['text'][bid]
parsed = extract_note_sections(full_text, common_headings)
for heading, content in parsed.items():
    print(f"== {heading} ==\n{content}\n")

== Chief Complaint: ==
Respiratory failure

== Major Surgical or Invasive Procedure: ==
None

== History of Present Illness: ==
HPI: Ms. [* * Known lastname 40526 * *] is a 61 year old female with history of cirrhosis due to HCV, type II DM, HTN, and GERD, who was recently discharged from [* * Hospital1 18 * *] on [* * 2164-11-17 * *] with SBP, who presents with altered mental status, sepsis and acute on chronic renal failure. She was discharged on ceftriaxone / vancomycin for SBP (SBP likely due to Klebsiella) and was also discharged on levaquin for aspiration PNA. On [* * 12-14 * *], she was found to have new onset seizures, and was transferred to [* * Hospital1 18 * *]. In the ED, initial vs were: T 98.4 HR 104 BP 105 / 50 RR 16 O2 sat 100%. Labs were notable for WBC 19.1, HCT 22, platelets 33, INR 2.4, lactate 4.5, AST 173, ALT 112, LDH 192, TBili 1.4, and Tbili 2.2. UA was positive for bacteria. CXR was notable for a left pleural effusion and a right lower lobe consolidation. She 

In [ ]:
if len(matches):
  parsed = extract_note_sections(matches['text'][0], common_headings, is_real=True)
  for heading, content in parsed.items():
      print(f"== {heading} ==\n{content}\n")
else:
  print("Cound not find any relevant real code set!")

== Chief Complaint: ==
Altered mental status and hypotension/Pneumonia

== Major Surgical or Invasive Procedure: ==
[**2130-5-15**]: Paracentesis
[**2130-5-16**]: Orthotopic Liver transplant
[**2130-5-23**]: Post pyloric feeding tube placement
[**2130-5-23**]: Flexible Bronchoscopy
[**2130-5-24**]: Pleural tap; ultrasound guided right pleuracentesis
[**2130-5-31**]: Post pyloric feeding tube placement

== History of Present Illness: ==
72 year old female with ESLD [**12-27**] HCV cirrhosis admitted to
MICU with AMS and sepsis.  Patient was noted to be minimaly
responsive by her family at home and brought to ED where whe was
febrile to 101.4 (103.2 rectal) and developed hypotension to
SBP=70s.  Pressors were intiated after IVF rescusitation failed
to improve her hypotension.  A paracentesis performed showing
4550 WBC with 72% PMNs. Vanco and zosyn were initiated. CXR
showing infiltrate as well.  AMS improved per family.  Overnight
neo was added to levophed for her hypotension.  She has 

## Evaluate Note Qualiaty

### Prepare

In [ ]:
!git clone https://github.com/chancholat/explain-icd.git
!pip install -q python-dotenv==1.0.0
!pip install -q datasets==3.4.1
!pip install -q omegaconf==2.3.0
!pip install -q captum==0.7.0
!pip install -q hydra-core
!pip install -q --force-reinstall transformers==4.38.1
!pip install -q --force-reinstall scikit-learn==1.3.0
!pip install -q --force-reinstall numpy==1.26.2

Cloning into 'explain-icd'...
remote: Enumerating objects: 1392, done.
remote: Counting objects: 100% (1392/1392), done.
remote: Compressing objects: 100% (677/677), done.
remote: Total 1392 (delta 743), reused 1349 (delta 700), pack-reused 0 (from 0)
Receiving objects: 100% (1392/1392), 4.37 MiB | 8.61 MiB/s, done.
Resolving deltas: 100% (743/743), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nv

In [ ]:
import kagglehub
#Download the latest version.
kagglehub.dataset_download('chanhainguyen/thesis-data-process')

100%|██████████| 2.95G/2.95G [00:50<00:00, 62.9MB/s]

Extracting files...


'/root/.cache/kagglehub/datasets/chanhainguyen/thesis-data-process/versions/1'

In [ ]:
!mv /root/.cache/kagglehub/datasets/chanhainguyen/thesis-data-process/versions/1/processed /content/explain-icd/data
# !mv /kaggle/input/thesis-data-process/processed /content/explain-icd/data

In [ ]:
# !gdown "https://drive.google.com/file/d/1TRDztpp6gVXR-MhUEPY3sAB8Vms2St-G" -O /content/explain-icd/data/processed/augmented_icd9_inpatient_code/train.parquet
!mkdir /content/explain-icd/data/processed/augmented_icd9_inpatient_code
!cp /content/explain-icd/data/processed/mdace_icd9_inpatient_code/test.parquet /content/explain-icd/data/processed/augmented_icd9_inpatient_code/
!cp /content/explain-icd/data/processed/mdace_icd9_inpatient_code/val.parquet /content/explain-icd/data/processed/augmented_icd9_inpatient_code/

In [ ]:
!mkdir /content/explain-icd/models
from huggingface_hub import snapshot_download

# Tải toàn bộ repository về thư mục local
snapshot_download(repo_id="ChanBeDu/PLM-ICD-seed10", repo_type="model", local_dir="/content/explain-icd/models")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

pgd/zauungyt/best_model.pt:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

igr/zmjo72tw/best_model.pt:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

config.yaml: 0.00B [00:00, ?B/s]

...

unsupervised/vxrn54op/best_model.pt:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

target_tokenizer.json: 0.00B [00:00, ?B/s]

config.yaml: 0.00B [00:00, ?B/s]

target_tokenizer.json: 0.00B [00:00, ?B/s]

KeyboardInterrupt: 

### Initialize

In [ ]:
import os

# Change to a specific directory
os.chdir("/content/explain-icd")

In [ ]:
# select model seed
run_id = 'unsupervised/vxrn54op'

#### Config init

In [ ]:
from hydra import initialize_config_dir, compose
import os

# Get the absolute path of the configuration directory
config_dir = os.path.abspath("explainable_medical_coding/config")

# with initialize_config_dir(config_dir=config_dir, job_name="test_app"):
#     cfg = compose(config_name="config", overrides=[
#         "experiment=mdace_icd9_code/plm_icd"
#     ])

with initialize_config_dir(config_dir=config_dir, job_name="test_app"):
    cfg = compose(config_name="config", overrides=[
        "experiment=augmented_icd9_code/plm_icd"
    ])

/tmp/ipython-input-577200088.py:12: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize_config_dir(config_dir=config_dir, job_name="test_app"):


In [ ]:
import torch
import os
import sys
import pandas as pd
from pathlib import Path
sys.path.append('/content/explain-icd')

from datasets import load_dataset
from omegaconf import OmegaConf
from transformers import AutoTokenizer

import explainable_medical_coding.config.factories as factories
from explainable_medical_coding.utils.loaders import (
    load_trained_model,
)
from explainable_medical_coding.utils.tokenizer import TargetTokenizer
from explainable_medical_coding.utils.analysis import predict
from explainable_medical_coding.utils.data_helper_functions import (
    create_targets_column,
    filter_unknown_targets,
    format_evidence_spans,
    get_unique_targets,
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/content/explain-icd/explainable_medical_coding/utils/loss_functions.py:13: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  diet_gradient_scaler = torch.cuda.amp.GradScaler()
/content/explain-icd/explainable_medical_coding/utils/loss_functions.py:14: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  advesarial_noise_gradient_scaler = torch.cuda.amp.GradScaler()


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if "CUDA_VISIBLE_DEVICES" not in os.environ:
  print("CUDA_VISIBLE_DEVICES not set")
  os.environ["CUDA_VISIBLE_DEVICES"] = "0"

CUDA_VISIBLE_DEVICES not set


In [ ]:
cfg.dataloader.batch_size = 8
cfg.dataloader.max_batch_size = 8

#### Dataset

In [ ]:
from datasets import load_dataset

# target_columns = list(cfg.data.target_columns)
# dataset_path = Path(cfg.data.dataset_path)
# model_path = Path(cfg.load_model) if cfg.load_model is not None else None
dataset_path = "/content/explain-icd/explainable_medical_coding/datasets/augmented_inpatient_icd9_code.py"
dataset = load_dataset(str(dataset_path))

The repository for augmented_inpatient_icd9_code contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/augmented_inpatient_icd9_code.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

##### Add informations to Augmeted dataset

In [ ]:
# from datasets import load_dataset

# dataset_path = '/content/explain-icd/explainable_medical_coding/datasets/mdace_inpatient_icd9_code.py'
# mimic = load_dataset(str(dataset_path), trust_remote_code=True)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
# # Please load mimic dataset, build ICD Hierarchy and selecting rare_codes first
# # All need codes in above sections
# def find_match_real_code_sets(augmented_code_set, mimic, rare_codes):
#   rare_codes_in_set = [code for code in augmented_code_set if code in rare_codes]
#   if not rare_codes_in_set:
#       return None
#   for rare_code in rare_codes_in_set:
#     match_entry = find_sibling_code_entry(mimic['train'], 'diagnosis_codes', set(augmented_code_set), rare_code)
#     if len(match_entry) > 0:
#       return match_entry
#   return None


# def extract_matched_procedure_codes(codes, mimic, rare_codes):
#     matched_row = find_match_real_code_sets(codes, mimic, rare_codes)
#     if matched_row is not None and len(matched_row) > 0:
#         return matched_row['procedure_codes'][0] # Assuming find_match_real_code_sets returns a Dataset with one row
#     else:
#         return []

# def extract_matched_note_id(codes, mimic, rare_codes):
#     matched_row = find_match_real_code_sets(codes, mimic, rare_codes)
#     if matched_row is not None and len(matched_row) > 0:
#         return matched_row['note_id']
#     else:
#         print(f"can not find match row for augmented codes {codes}")
#         return []

# # # Update 'procedure_codes' column by mapping row-by-row
# # dataset['train'] = dataset['train'].map(
# #     lambda example: {
# #         'procedure_codes': extract_matched_procedure_codes(example['diagnosis_codes'], mimic, rare_codes)
# #     },
# #     num_proc=2,
# # )

In [ ]:
# def flatten_procedure_codes(example):
#     flat_list = [code for sublist in example['procedure_codes'] for code in sublist]
#     return {'procedure_codes': flat_list}
# dataset['train'] = dataset['train'].map(flatten_procedure_codes, num_proc=2)

In [ ]:
# from huggingface_hub import login

# login(token="")  # Get this from https://huggingface.co/settings/tokens
# dataset.push_to_hub("ChanBeDu/augmented_icd9_inpatent_code")

In [ ]:
# # get note_id column by mapping row-by-row
# match_note_ids = dataset['train'].map(
#     lambda example: {
#         'match_note_id': extract_matched_note_id(example['diagnosis_codes'], mimic, rare_codes)
#     },
#     num_proc=4,
# )

Map (num_proc=4):   0%|          | 0/3019 [00:00<?, ? examples/s]

In [ ]:
# match_note_ids['match_note_id']

In [ ]:
# from huggingface_hub import login

# login(token="")  # Get this from https://huggingface.co/settings/tokens
# match_note_ids.push_to_hub("ChanBeDu/augmented_icd9_note_id_full")

##### Extract real relevant dataset

In [ ]:
from datasets import load_dataset

# Load the dataset
augmented_note_id = load_dataset("ChanBeDu/augmented_icd9_note_id")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/864 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/865 [00:00<?, ? examples/s]

In [ ]:
dataset_path = '/content/explain-icd/explainable_medical_coding/datasets/mdace_inpatient_icd9_code.py'
mimic = load_dataset(str(dataset_path), trust_remote_code=True)

In [ ]:
# Extract note_ids from dataset_a
note_ids_to_keep = set([ids[0] for ids in augmented_note_id['train']["match_note_id"] if ids] )

# Filter dataset_b to keep only rows with note_id in dataset_a
dataset['train'] = mimic['train'].filter(lambda x: x["note_id"] in note_ids_to_keep)

# Check result
print(len(dataset['train']))

Filter:   0%|          | 0/48074 [00:00<?, ? examples/s]

755


#### Tokenize dataset

In [ ]:
from explainable_medical_coding.utils.settings import TARGET_COLUMN, TEXT_COLUMN

text_tokenizer = AutoTokenizer.from_pretrained(cfg.model.configs.model_path,)
max_input_length = int(cfg.data.max_length)
# tokenize text
dataset = dataset.map(
    lambda x: text_tokenizer(
        x[TEXT_COLUMN],
        return_length=True,
        truncation=True,
        max_length=max_input_length,
    ),
    batched=True,
    # num_proc=8,
    num_proc=4,
    batch_size=1_000,
    desc="Tokenizing text",
)

Tokenizing text (num_proc=4):   0%|          | 0/755 [00:00<?, ? examples/s]

Tokenizing text (num_proc=4):   0%|          | 0/1753 [00:00<?, ? examples/s]

Tokenizing text (num_proc=4):   0%|          | 0/3197 [00:00<?, ? examples/s]

#### Model and trainer

In [ ]:
import logging
LOGGER = logging.getLogger(name="test evaluate")
LOGGER.setLevel(logging.INFO)

model_folder_path = Path('models')
model_path = model_folder_path / run_id

In [ ]:
dataset = dataset.map(
    lambda x: create_targets_column(x, target_columns),
    desc="Creating targets column",
)
known_targets = set(get_unique_targets(dataset))
dataset = dataset.map(
    lambda x: filter_unknown_targets(x, known_targets=known_targets),
    desc="Filter unknown targets",
)
dataset = dataset.filter(
    lambda x: len(x[TARGET_COLUMN]) > 0, desc="Filtering empty targets"
)
dataset = dataset.map(lambda x: format_evidence_spans(x, text_tokenizer))

autoregressive = bool(cfg.model.autoregressive)
target_tokenizer = TargetTokenizer(autoregressive=autoregressive)
if model_path is None:
    unique_targets = get_unique_targets(dataset)
    target_tokenizer.fit(unique_targets)
else:
    LOGGER.info("Loading Tokenizer from model_path")
    target_tokenizer.load(model_path / "target_tokenizer.json")

# convert targets to target ids
dataset = dataset.map(
    lambda x: {"target_ids": target_tokenizer(x[TARGET_COLUMN])},
    desc="Converting targets to target ids",
)
dataset.set_format(
    type="torch", columns=["input_ids", "length", "attention_mask", "target_ids"]
)

Creating targets column:   0%|          | 0/755 [00:00<?, ? examples/s]

Creating targets column:   0%|          | 0/1753 [00:00<?, ? examples/s]

Creating targets column:   0%|          | 0/3197 [00:00<?, ? examples/s]

Filter unknown targets:   0%|          | 0/755 [00:00<?, ? examples/s]

Filter unknown targets:   0%|          | 0/1753 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/3197 [00:00<?, ? examples/s]

INFO:test evaluate:Loading Tokenizer from model_path


Converting targets to target ids:   0%|          | 0/755 [00:00<?, ? examples/s]

Converting targets to target ids:   0%|          | 0/1753 [00:00<?, ? examples/s]

Converting targets to target ids:   0%|          | 0/3197 [00:00<?, ? examples/s]

In [ ]:
lookups = factories.get_lookups(
      dataset=dataset,
      text_tokenizer=text_tokenizer,
      target_tokenizer=target_tokenizer,
  )
LOGGER.info(lookups.data_info)

/content/explain-icd/explainable_medical_coding/utils/lookups.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  split2code_indices[split_name] = torch.tensor(target_ids)
INFO:test evaluate:{'num_examples': 5705, 'num_train_examples': 755, 'num_val_examples': 1753, 'num_test_examples': 3197, 'average_words_per_example': 3865.7875178584814, 'average_targets_per_example': 18.490680298738923, 'num_classes': 8943, 'num_train_classes': 2490, 'num_val_classes': 3044, 'num_test_classes': 3980, 'vocab_size': 50001, 'pad_token_id': 1, 'pad_target_id': -1, 'sos_target_id': None, 'eos_target_id': None}


In [ ]:
if model_path is None:
    model = factories.get_model(config=cfg.model, data_info=lookups.data_info)
else:
    LOGGER.info("Loading Model from model_path")
    saved_config = OmegaConf.load(model_path / "config.yaml")
    model, decision_boundary = load_trained_model(
        model_path,
        saved_config,
        pad_token_id=text_tokenizer.pad_token_id,
        device=device,
    )

model.to(device)

INFO:test evaluate:Loading Model from model_path


{'name': 'PLMICD', 'autoregressive': False, 'configs': {'model_path': 'models/roberta-base-pm-m3-voc-hf', 'chunk_size': 128, 'cross_attention': True, 'loss': 'binary_cross_entropy', 'lambda_1': 0.0, 'scale': 1, 'mask_input': False}}


PLMICD(
  (roberta_encoder): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50008, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Laye

In [ ]:
import math

loss_function = factories.get_loss_function(config=cfg.loss)

dataloaders = factories.get_dataloaders(
    config=cfg.dataloader,
    dataset=dataset,
    target_tokenizer=lookups.target_tokenizer,
    pad_token_id=lookups.data_info["pad_token_id"],
)

metric_collections = factories.get_metric_collections(
    config=cfg.metrics,
    number_of_classes=lookups.data_info["num_classes"],
    split2code_indices=lookups.split2code_indices,
    autoregressive=cfg.model.autoregressive,
)

optimizer = factories.get_optimizer(config=cfg.optimizer, model=model)
accumulate_grad_batches = int(
    max(cfg.dataloader.batch_size / cfg.dataloader.max_batch_size, 1)
)
num_training_steps = (
    math.ceil(len(dataloaders["train"]) / accumulate_grad_batches)
    * cfg.trainer.epochs
)
lr_scheduler = factories.get_lr_scheduler(
    config=cfg.lr_scheduler,
    optimizer=optimizer,
    num_training_steps=num_training_steps,
)
callbacks = factories.get_callbacks(config=cfg.callbacks)
trainer_class = factories.get_trainer(name=cfg.trainer.name)
trainer = trainer_class(
    config=cfg,
    model=model,
    loss_function=loss_function,
    optimizer=optimizer,
    dataloaders=dataloaders,
    metric_collections=metric_collections,
    callbacks=callbacks,
    lr_scheduler=lr_scheduler,
    lookups=lookups,
    accumulate_grad_batches=accumulate_grad_batches,
).to(device)

'Train batch size: 8'

'Accumulating gradients over 1 batch(es).'

/content/explain-icd/explainable_medical_coding/trainer/trainer.py:55: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.gradient_scaler = torch.cuda.amp.GradScaler(enabled=self.use_amp)
wandb: WARNING `start_method` is deprecated and will be removed in a future version of wandb. This setting is currently non-functional and safely ignored.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: chanhainguyen97 (chanhainguyen97-University of Science) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


### Evaluate

In [ ]:
trainer.val(trainer.epoch, "train", evaluating_best_model=True)
# trainer.val(trainer.epoch, "validation", evaluating_best_model=True)
# trainer.val(trainer.epoch, "test", evaluating_best_model=True)

Output()

In [ ]:
trainer.metric_collections['train'].threshold

0.5

In [ ]:
f1_macro = [m for m in trainer.metric_collections['train'].metrics if m.name == 'f1_macro'][0]
f1_macro.compute()

tensor(0.3310)

In [ ]:
f1_micro = [m for m in trainer.metric_collections['train'].metrics if m.name == 'f1_micro'][0]
f1_micro.compute()

tensor(0.6160)

In [ ]:
import torch
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [ ]:
from google.colab import files

def download_result(file_path):
  # Download the file
  files.download(file_path)

# train
download_result("/content/explain-icd/models/zgir9xnt/s10-real-relevant_prediction_train.feather")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Analyse

#### Tunning decision boundary

In [ ]:
import torch
import pandas as pd
from typing import Optional
from sklearn.preprocessing import MultiLabelBinarizer
from explainable_medical_coding.utils.decision_boundary import f1_score_db_tuning

def filter_tensor(
    tensor: torch.Tensor, code_indices: Optional[torch.Tensor]
) -> torch.Tensor:
    if code_indices is None:
        return tensor
    return torch.index_select(tensor, -1, code_indices)


def tunning_f1(df, average, code_indices: Optional[torch.Tensor], rare_sets = [], is_on_rare = False):

  logit_columns = [col for col in df.columns if col not in ['target', '_id']]
  if is_on_rare:
    logit_columns = [col for col in logit_columns if col in rare_sets]

  logits_df = df[logit_columns]
  logits = torch.tensor(logits_df.values, dtype=torch.float32)
  logits = filter_tensor(logits, code_indices)
  # Use MultiLabelBinarizer to create the one-hot encoded target matrix
  mlb = MultiLabelBinarizer(classes=logit_columns)
  # The target column contains lists of codes, which is the expected input for fit_transform
  targets_one_hot = mlb.fit_transform(df['target'])

  # Convert the one-hot encoded targets to a PyTorch tensor
  targets = torch.tensor(targets_one_hot, dtype=torch.long)
  targets = filter_tensor(targets, code_indices)
  # Move tensors to the same device if you are using CUDA
  if torch.cuda.is_available():
      device = torch.device("cuda")
      logits = logits.to(device)
      targets = targets.to(device)
  else:
      device = torch.device("cpu")

  # Now you can use the f1_score_db_tuning function
  best_f1, best_db = f1_score_db_tuning(logits, targets, average=average)

  print(f"\nUsing f1_score_db_tuning:")
  print(f"Best {average} F1 Score: {best_f1}")
  print(f"Best Decision Boundary: {best_db}")
  return best_f1, best_db

In [ ]:
prediction_df = pd.read_feather('/content/explain-icd/models/zgir9xnt/s10-real-relevant_prediction_train.feather')
code_indices = lookups.split2code_indices['train']
best_f1, best_db = tunning_f1(prediction_df, 'macro', code_indices)

Best F1: 0.46432819962501526 at DB: 0.10101009905338287

Using f1_score_db_tuning:
Best macro F1 Score: 0.46432819962501526
Best Decision Boundary: 0.10101009905338287


In [ ]:
prediction_df = pd.read_feather('/content/explain-icd/models/zgir9xnt/s10-real-relevant_prediction_train.feather')
code_indices = lookups.split2code_indices['train']
best_f1, best_db = tunning_f1(prediction_df, 'micro', code_indices)

Best F1: 0.6428772807121277 at DB: 0.3232323229312897

Using f1_score_db_tuning:
Best micro F1 Score: 0.6428772807121277
Best Decision Boundary: 0.3232323229312897


#### Show result

In [ ]:
def add_multilabel_predictions(df: pd.DataFrame, class_columns: list, threshold: float = 0.5, fallback_to_max: bool = False) -> None:
    """
    Adds a 'predicted_class' column for multi-label classification based on a probability threshold.

    Parameters:
    - df: DataFrame with class probability columns.
    - class_columns: List of class probability column names.
    - threshold: Minimum probability to consider a class as predicted.
    - fallback_to_max: If True, return the top class if none exceed threshold; otherwise return [].
    """
    def predict_row(row):
        probs = row[class_columns]
        selected = probs[probs >= threshold].index.tolist()
        if selected:
            return selected
        return [probs.idxmax()] if fallback_to_max else []

    df["predicted_class"] = df.apply(predict_row, axis=1)


In [ ]:
best_db.item()

0.020202020183205605

In [ ]:
class_cols = [col for col in prediction_df.columns if col not in ['target', '_id']]
add_multilabel_predictions(prediction_df, class_cols, best_db.item())

In [ ]:
prediction_df[['target', 'predicted_class']].head()

,target,predicted_class
0,"[401.9, 428.0, 250.00, 410.11, 584.9, V58.67, ...","[038.9, 410.71, 410.72, 37.21, 790.29, 410.01,..."
1,"[891.0, 860.0, 710.1, 041.4, 577.2, 599.0, E81...","[807.03, E849.5, 852.20, 518.5, 860.0, 807.08,..."
2,"[403.90, 710.0, E870.8, 707.21, 428.32, 585.9,...","[710.0, 425.4, 338.29, 428.23, 583.81, 428.22,..."
3,"[821.01, 593.2, 518.5, 401.9, 250.40, 584.5, 8...","[E849.5, 038.9, 518.5, 458.29, 287.5, 401.9, 2..."
4,"[452, 288.00, 724.5, 572.3, 571.5, 455.5, 507....","[42.81, 530.3, 535.10, 38.97, 070.44, 276.3, 4..."


### Get Synthesis prediction from hub

In [ ]:
!mkdir /content/explain-icd/models
!mkdir /content/explain-icd/models-synthesis

In [ ]:
from huggingface_hub import snapshot_download

# Tải toàn bộ repository về thư mục local
snapshot_download(repo_id="ChanBeDu/Synthesis-PLM-ICD", repo_type="model", local_dir="/content/explain-icd/models-synthesis")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

best_model.pt:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

best_model.pt:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

predictions_test.feather:   0%|          | 0.00/118M [00:00<?, ?B/s]

...

predictions_test.feather:   0%|          | 0.00/118M [00:00<?, ?B/s]

config.yaml: 0.00B [00:00, ?B/s]

predictions_validation.feather:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

target_tokenizer.json: 0.00B [00:00, ?B/s]

'/content/explain-icd/models'